First project setup to running SetupPythonVirtualEnvirontment.bat

1. How To connection with the Elasticsearch 

In [ ]:
# Simple connection to elasticsearch 
from pprint import pprint
from elasticsearch import Elasticsearch
from config.development import Config

es = Elasticsearch(
    Config.ES_HOST, 
    verify_certs=True,
    basic_auth = (Config.ES_USERNAME, Config.ES_PASSWORD)
)


client_info = es.info()
print("Success connected to elastic client ✔")
pprint(client_info.body)

ModuleNotFoundError: No module named 'config'

In [5]:
from config.development import Config


ModuleNotFoundError: No module named 'config'

2. Tutorial to Create and Index with or Without Shard and replicas

In [2]:
# Create and Index
# 1. Simple way to create an index 
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [2]:
# 2. Specify the Number Of replicas and shards
# shards : Elasticsearch is divides the data into mutiple shards. Each shard is a self-contained index that Elasticsearch can distribute accross multiple nodes in a cluster. 
# Shards are managed automatically but configured when creating the index. 
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(
    index='my_index', 
    settings={
        "index": {
            "number_of_shards": 3, 
            "number_of_replicas": 2
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

4. How to Inserting documents into index in elasticsearch. Explain what is document in elasticsearch.

In [10]:
# 1. Create a dummy index first
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [12]:
from pprint import pprint
# 2. Create a dummy document with 3 fields
document = {
    "content": "Content of the pages",
    "title": "This is title", 
    "pages": "50", 
    "authors": "John Wick", 
    "created_on": "2025-10-20"
}

response = es.index(index="my_index", body=document)
pprint(response.body)
print ("---------------------------")
# response_data = es.get(index="my_index")
# # pprint(response_data.body)

{'_id': 'tVyl65kB3GCK3pvL2fu8',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': '_doc',
 '_version': 1,
 'result': 'created'}
---------------------------


The response object contains the result of operation. If we successfully inserted the document result = created


In [ ]:
response = es.get(index="my_index", id=[0])
print(response["result"])

NotFoundError: NotFoundError(404, "{'_index': 'my_index', '_type': '_doc', '_id': '0', 'found': False}")

In [87]:
print(response['_shards'])

{'total': 2, 'successful': 1, 'failed': 0}


In [88]:
print(response["_id"])

S68O5pkBUuND_Nx8pHxG


In [89]:
print(response["_index"])

my_index


Insert multiple documents. Just do the same step but i a for loop

In [90]:
# Create 3 dummy json file
import json

dummy_data = json.load(open("../data/dummy_data.json"))
dummy_data

[{'title': 'Sample title 1',
  'content': 'This is sample content first entry',
  'created_on': '2023-10-01'},
 {'title': 'Sample title 2',
  'content': 'This is sample content second enty',
  'created_on': '2023-10-02'},
 {'title': 'Sample title 3',
  'content': 'This is sample content third entry',
  'created_on': '2023-10-03'}]

In [91]:
# Method to insert data in the document
def insert_document(document): 
    response = es.index(index="my_index", body=document)
    return response

# Method for print the response
def print_info(response): 
    print(f"""Document ID: {response['_id']} is '{
        response["result"]}' and is split into {response['_shards']['total']} shards.""")
    
    
# For loop docuent
for document in dummy_data:
    response = insert_document(document)
    print_info(response)

Document ID: TK8O5pkBUuND_Nx8pHzk is 'created' and is split into 2 shards.
Document ID: Ta8O5pkBUuND_Nx8pXwh is 'created' and is split into 2 shards.
Document ID: Tq8O5pkBUuND_Nx8pXw6 is 'created' and is split into 2 shards.


How to print the mapping in elasticsearch 

In [92]:
from pprint import pprint

index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping["my_index"]["mappings"]["properties"])

{'content': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
             'type': 'text'},
 'created_on': {'type': 'date'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}


Using a manual mapping

In [93]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

# How to manualy mapping 
mapping = {
    'properties': {
        'created_on': {'type': 'date'},
        'text': {
            'type': 'text', 
            'fields': {
                'keyword': {
                    'type': 'keyword', 
                    'ignore_above': 256
                }
            }
        }, 
        'title': {
            'type': 'text', 
            'fields': {
                'keyword': {
                    'type': 'keyword',
                     'ignore_above': 256
                }
            }
            
        }
    }
}

es.indices.put_mapping(index='my_index', body=mapping)

index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping["my_index"]["mappings"]["properties"])

{'created_on': {'type': 'date'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}


In [94]:
# Tutorial How to make a mapping elasticsearch 
mapping = {
    'properties': {
        'created_on': {'type': 'date'}, 
        'text': {
            'type': 'text', 
            'fields': {
                'keyword': {
                    'type': 'keyword', 
                    'ignore_above': 256
                }
            }
        }, 
        'title': {
            'type': 'text', 
            'fields': {
                'keyword': {
                    'type': 'keyword', 
                    'ignore_above': 256
                }
            }
        }
    }
}

# Delete the index
es.indices.delete(index='my_index', ignore_unavailable=True)

# Create the index
es.indices.create(
    index='my_index', 
    mappings=mapping)

index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping["my_index"]["mappings"]["properties"])

{'created_on': {'type': 'date'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}


5. common types -> how to use data types

In [95]:
# 1.1 binary 
es.indices.delete(index='binary_index', ignore_unavailable=True)

# Create index
es.indices.create(
    index='binary_index', 
    mappings={
        "properties": {
            "image_data": {
                "type": 'binary'
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'binary_index'})

In [96]:
import base64

image_path = '../images/about-lombok.jpg'
with open(image_path, "rb") as image_file:
    image_bytes = image_file.read()
    image_base64 = base64.b64encode(image_bytes).decode("utf-8")
    
    
image_base64[:100]

'UklGRqT6AQBXRUJQVlA4IJj6AQBwyAedASpMBCADPm0ylEemJCKhp5da4MANiWNt5+leP7rE1/k6pv6u5uEAwh/1Oguz256cqyaf'

In [97]:
len(image_base64)

172944

In [98]:
document = {
    "image_data" : image_base64
}

response = es.index(index='binary_index', body=document)
response

ObjectApiResponse({'_index': 'binary_index', '_type': '_doc', '_id': 'T68O5pkBUuND_Nx8qHzi', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

1.2 Others

In [99]:
es.indices.delete(
    index='other_common_data_types_index', 
    ignore_unavailable=True
)

# Make a mapping index
es.indices.create(
    index='other_common_data_types_index', 
    mappings={
        "properties": {
            "book_preference": {
                "type": "keyword"
            }, 
            "price": {
                "type": "float"
            }, 
            "publish_date": {
                "type": "date"
            }, 
            "is_available": {
                "type": "date"
            }
        }
    }
)



ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'other_common_data_types_index'})

In [100]:
# Insert data to document
document = {
    "book_preference": "9723-2834824", 
    "price": 33.23, 
    "publish_date": "2021-06-30", 
    "is_available": True
}

response = es.index(index='other_common_datat_types_index', body=document)
response


ObjectApiResponse({'_index': 'other_common_datat_types_index', '_type': '_doc', '_id': 'UK8O5pkBUuND_Nx8qXzw', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

2.1 Object Types Of Index

2.1 Object Index

In [101]:
# 2.1 Object 
es.indices.delete(index='object_index', ignore_unavailable=True)

# Create a mapping nested object 
es.indices.create(
    index='object_index', 
    mappings={
        "properties": {
            "author": {
                "properties": {
                    "first_name": {
                        "type": "text"
                    }, 
                    "last_name": {
                        "type": "text"
                    }
                }
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'object_index'})

In [102]:
# make an dummy data 
document = {
    "author": {
        "first_name": "Imam", 
        "last_name": "Sadikid"
    }
}

response = es.index(index='object_index', body=document)
response

ObjectApiResponse({'_index': 'object_index', '_type': '_doc', '_id': 'Ua8O5pkBUuND_Nx8qnz7', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

2.2 Flattened Object Index

In [103]:
# 2.2 How To use a flattened object 
es.indices.delete(
    index="flattened_object_index", 
    ignore_unavailable=True
)

# Make index mapping with flattened 
es.indices.create(
    index="flattened_object_index", 
    mappings={
        "properties": {
            "author": {
                "type": "flattened"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'flattened_object_index'})

In [104]:
# Make an insert document data
document = {
    "author": {
        "first_name": "Imam", 
        "last_name": "sadikik"
    }
}

response = es.index(
    index="flattened_object_index", 
    body=document
    )

response

ObjectApiResponse({'_index': 'flattened_object_index', '_type': '_doc', '_id': 'Uq8O5pkBUuND_Nx8rHwR', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

2.3 Using a nested document

In [105]:
es.indices.delete(
    index='nested_object_index', 
    ignore_unavailable=True
)

es.indices.create(
    index="nested_object_index", 
    mappings={
        "properties": {
            "users": {
                "type": "nested"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'nested_object_index'})

In [106]:
# Insert data 
documents = [
    {
        "first": "John", 
        "last": "Wick"
    }, 
    {
        "first": "Imam", 
        "last": "Sadikik"
    }
]

response = es.index(
    index="nested_object_index", 
    body={"user": document}
)

response

ObjectApiResponse({'_index': 'nested_object_index', '_type': '_doc', '_id': 'U68O5pkBUuND_Nx8rXwL', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

3. Text Search Types

In [107]:
# Delete index first 
es.indices.delete(index="text_index", ignore_unavailable=True)

# Make an index
es.indices.create(
    index="text_index", 
    mappings={
        "properties": {
            "email_body": {
                "type": "text"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'text_index'})

In [110]:
# Make a document inserted 
document = {
    "email_body": "Hello, this is a test email."
}

response = es.index(index="email_body", body=document)
response

ObjectApiResponse({'_index': 'email_body', '_type': '_doc', '_id': 'WK8X5pkBUuND_Nx8t3yg', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1})

------ 3.2 Suggest : "completion" type

In [111]:
# 3.2 suggest type of completion
es.indices.delete(index="text_completion_index", ignore_unavailable=True)
es.indices.create(
    index="text_completion_index", 
    mappings={
        "properties": {
            "suggest": {
                "type": "completion"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'text_completion_index'})

In [112]:
# make an input data 
document_1 = {
    "suggest": {
        "input": ["Merkurius", "Venus", "Mars"]
    }
}

document_2 = {
    "sugges": {
        "input": ["Galaxy", "Andromeda"]
    }
}

es.index(index="text_completion_index", body=document_1)
es.index(index="text-completion_index", body=document_2)

ObjectApiResponse({'_index': 'text-completion_index', '_type': '_doc', '_id': 'W68d5pkBUuND_Nx8V3zy', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

4. Spatial Data Types

4.1 geo_point type

In [138]:
# Make a geopoint index with type ge
es.indices.delete(index="geo_point_index", ignore_unavailable=True)
es.indices.create(
    index="geo_point_index",
    mappings={
        "properties": {
            "location": {
                "type": "geo_point"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_point_index'})

In [141]:
# Make the inserted document 
document = {
    "text": "Geopoint as an object using GeoJSON Format", 
    "location": [
        -71.5, 23.4
    ]    
}

response = es.index(index="geo_point_index", body=document)
response

ObjectApiResponse({'_index': 'geo_point_index', '_type': '_doc', '_id': 'eq-G5pkBUuND_Nx8tnyC', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

4.2 geo_shape type 

In [162]:
es.indices.delete(index="geo_shape_index", ignore_unavailable=True)
es.indices.create(
    index="geo_shape_index", 
    mappings={
        "properties": {
            "location": {
                "type": "geo_shape"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_shape_index'})

In [163]:
# Make a insert data 
document_1 = {
    "location": {
        "type": "LineString", 
        "coordinates": [
            [
                -22.5, 
                32.5
            ], 
            [
                -32.75, 
                20.2
            ]
        ]
    }
}

document_2 = {
    "location": {
        "type": "Polygon", 
        "coordinates": [
            [
                [100, 0], 
                [104, 0], 
                [104, 1], 
                [100, 1], 
                [100, 0]  # closed ring
            ], 
            [
                [102.3, 0.2], 
                [102.3, 3.1], 
                [102.3, 2.1], 
                [102.3, 1.2], 
                [102.3, 0.2]  # closed ring
            ]
        ]
    }
}


es.index(index="geo_shape_index", body=document_1)
es.index(index="geo_shape_index", body=document_2)

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'failed to parse field [location] of type [geo_shape]', at least 4 polygon points required)

4.3 Point

In [164]:
es.indices.delete(index='point_index', ignore_unavailable=True)
es.indices.create(
    index="point_index", 
    mappings={
        "properties": {
            "location": {
                "type": "point"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'point_index'})

In [166]:
document = {
    "location": [
        -71.4, 
        21.4
    ]
}

response = es.index(index="point_index", body=document)
response

ObjectApiResponse({'_index': 'point_index', '_type': '_doc', '_id': 'nK-m5pkBUuND_Nx85Hwj', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

6. Delete Document Index in Elasticseaserch